<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Stream Classification
---

## `NEWeather` dataset

**Description:** The National Oceanic and Atmospheric Administration (NOAA),
has compiled a database of weather measurements from over 7,000 weather 
stations worldwide. Records date back to the mid-1900s. Daily measurements
include a variety of features (temperature, pressure, wind speed, etc.) as
well as a series of indicators for precipitation and other weather-related
events. The `NEweather` dataset contains data from this database, specifically
from the Offutt Air Force Base in Bellevue, Nebraska ranging for over 50 years
(1949-1999).

**Features:** 8 Daily weather measurements
 
|       Attribute      | Description |
|:--------------------:|:-----------------------------|
| `temp`                   | Temperature
| `dew_pnt`                | Dew Point
| `sea_lvl_press`          | Sea Level Pressure
| `visibility`             | Visibility
| `avg_wind_spd`           | Average Wind Speed
| `max_sustained_wind_spd` | Maximum Sustained Wind Speed
| `max_temp`               | Maximum Temperature
| `min_temp`               | Minimum Temperature


**Class:** `rain` | 0: no rain, 1: rain
 
**Samples:** 18,159


In [1]:
import pandas as pd
from river.stream import iter_pandas
from river.metrics import Rolling,Metrics,Accuracy,BalancedAccuracy,CohenKappa,GeometricMean
from river.evaluate import progressive_val_score

In [2]:
data = pd.read_csv("../datasets/NEweather.csv")
features = data.columns[:-1]

In this example, we load the data from a csv file with `pandas.read_csv`, and we use the [iter_pandas](https://riverml.xyz/latest/api/stream/iter-pandas/) utility method to iterate over the `DataFrame`.

In [3]:
stream = iter_pandas(X=data[features], y=data['rain'])

## Naïve Bayes
---
[GaussianNB](https://riverml.xyz/latest/api/naive-bayes/GaussianNB/) maintains a Gaussian distribution $G_{cf}$ is maintained for each class $c$ and each feature $f$. Each Gaussian is updated using the amount associated with each feature; the details can be be found in proba.Gaussian. The joint log-likelihood is then obtained by summing the log probabilities of each feature associated with each class.

In [4]:
from river.naive_bayes import GaussianNB

model = GaussianNB()
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 71.27%, BalancedAccuracy: 71.96%, GeometricMean: 71.93%, CohenKappa: 40.19%
[2,000] Accuracy: 69.88%, BalancedAccuracy: 70.38%, GeometricMean: 70.37%, CohenKappa: 36.23%
[3,000] Accuracy: 68.99%, BalancedAccuracy: 69.64%, GeometricMean: 69.62%, CohenKappa: 34.23%
[4,000] Accuracy: 68.82%, BalancedAccuracy: 68.87%, GeometricMean: 68.87%, CohenKappa: 33.48%
[5,000] Accuracy: 69.09%, BalancedAccuracy: 67.97%, GeometricMean: 67.92%, CohenKappa: 32.70%
[6,000] Accuracy: 69.13%, BalancedAccuracy: 67.87%, GeometricMean: 67.80%, CohenKappa: 32.65%
[7,000] Accuracy: 69.15%, BalancedAccuracy: 67.89%, GeometricMean: 67.82%, CohenKappa: 32.62%
[8,000] Accuracy: 68.50%, BalancedAccuracy: 67.31%, GeometricMean: 67.25%, CohenKappa: 31.56%
[9,000] Accuracy: 68.65%, BalancedAccuracy: 66.69%, GeometricMean: 66.50%, CohenKappa: 30.97%
[10,000] Accuracy: 69.04%, BalancedAccuracy: 66.36%, GeometricMean: 66.01%, CohenKappa: 30.75%
[11,000] Accuracy: 69.52%, BalancedAccuracy: 66.52%, Geomet

Accuracy: 69.21%, BalancedAccuracy: 66.27%, GeometricMean: 65.80%, CohenKappa: 31.28%

## K-Nearest Neighbors
---
[KNN](https://riverml.xyz/latest/api/neighbors/KNNClassifier/) is a non-parametric classification method that keeps track of the last window_size training samples. The predicted class-label for a given query sample is obtained in two steps:

- Find the closest n_neighbors to the query sample in the data window. 
- Aggregate the class-labels of the n_neighbors to define the predicted class for the query sample.

In [5]:
from river.neighbors import KNNClassifier

model = KNNClassifier(n_neighbors=5, window_size=1000)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 77.18%, BalancedAccuracy: 71.52%, GeometricMean: 69.63%, CohenKappa: 45.32%
[2,000] Accuracy: 78.34%, BalancedAccuracy: 71.07%, GeometricMean: 68.71%, CohenKappa: 44.95%
[3,000] Accuracy: 78.86%, BalancedAccuracy: 70.49%, GeometricMean: 67.63%, CohenKappa: 44.34%
[4,000] Accuracy: 78.29%, BalancedAccuracy: 70.44%, GeometricMean: 67.78%, CohenKappa: 43.93%
[5,000] Accuracy: 78.06%, BalancedAccuracy: 70.37%, GeometricMean: 67.77%, CohenKappa: 43.68%
[6,000] Accuracy: 77.95%, BalancedAccuracy: 70.53%, GeometricMean: 68.08%, CohenKappa: 43.80%
[7,000] Accuracy: 78.24%, BalancedAccuracy: 70.95%, GeometricMean: 68.65%, CohenKappa: 44.55%
[8,000] Accuracy: 77.96%, BalancedAccuracy: 70.85%, GeometricMean: 68.60%, CohenKappa: 44.26%
[9,000] Accuracy: 78.12%, BalancedAccuracy: 71.08%, GeometricMean: 68.83%, CohenKappa: 44.81%
[10,000] Accuracy: 78.16%, BalancedAccuracy: 71.12%, GeometricMean: 68.91%, CohenKappa: 44.84%
[11,000] Accuracy: 78.35%, BalancedAccuracy: 71.33%, Geomet

Accuracy: 77.91%, BalancedAccuracy: 72.06%, GeometricMean: 70.33%, CohenKappa: 46.24%

## Hoeffding Tree
---

[Hoeffding Tree](https://riverml.xyz/latest/api/tree/HoeffdingTreeClassifier/) 

Tree-based models are popular due to their interpretability. They use a tree data structure to model the data. When a sample arrives, it traverses the tree until it reaches a leaf node. Internal nodes define the path for a data sample based on the values of its features. Leaf nodes are models that provide predictions for unlabeled-samples and can update their internal state using the labels from labeled samples.

In [6]:
from river.tree import HoeffdingTreeClassifier

model = HoeffdingTreeClassifier()
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=1000)

[1,000] Accuracy: 70.87%, BalancedAccuracy: 71.11%, GeometricMean: 71.10%, CohenKappa: 38.92%
[2,000] Accuracy: 69.73%, BalancedAccuracy: 68.12%, GeometricMean: 68.01%, CohenKappa: 33.45%
[3,000] Accuracy: 70.89%, BalancedAccuracy: 63.00%, GeometricMean: 60.16%, CohenKappa: 26.85%
[4,000] Accuracy: 71.29%, BalancedAccuracy: 61.85%, GeometricMean: 57.40%, CohenKappa: 25.57%
[5,000] Accuracy: 71.79%, BalancedAccuracy: 62.23%, GeometricMean: 57.58%, CohenKappa: 26.59%
[6,000] Accuracy: 72.13%, BalancedAccuracy: 62.56%, GeometricMean: 57.88%, CohenKappa: 27.40%
[7,000] Accuracy: 72.82%, BalancedAccuracy: 64.11%, GeometricMean: 60.42%, CohenKappa: 30.23%
[8,000] Accuracy: 72.58%, BalancedAccuracy: 64.31%, GeometricMean: 60.90%, CohenKappa: 30.45%
[9,000] Accuracy: 72.80%, BalancedAccuracy: 63.98%, GeometricMean: 59.98%, CohenKappa: 30.21%
[10,000] Accuracy: 72.85%, BalancedAccuracy: 63.64%, GeometricMean: 59.32%, CohenKappa: 29.69%
[11,000] Accuracy: 73.30%, BalancedAccuracy: 63.81%, Geomet

Accuracy: 73.55%, BalancedAccuracy: 65.87%, GeometricMean: 62.56%, CohenKappa: 34.07%

## Hoeffding Adaptive Tree
---
The [HAT](https://riverml.xyz/latest/api/tree/HoeffdingAdaptiveTreeClassifier/) model uses `ADWIN` to detect changes. If change is detected in a given branch, an alternate branch is created and eventually replaces the original branch if it shows better performance on new data.

In [7]:
from river.tree import HoeffdingAdaptiveTreeClassifier

model = HoeffdingAdaptiveTreeClassifier(seed=42)
metrics = Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()])
stream = iter_pandas(X=data[features], y=data['rain'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metrics, 
                      print_every=1000)

[1,000] Accuracy: 68.37%, BalancedAccuracy: 65.29%, GeometricMean: 64.69%, CohenKappa: 29.83%
[2,000] Accuracy: 69.48%, BalancedAccuracy: 63.36%, GeometricMean: 61.49%, CohenKappa: 26.88%
[3,000] Accuracy: 71.09%, BalancedAccuracy: 60.64%, GeometricMean: 55.35%, CohenKappa: 23.23%
[4,000] Accuracy: 72.02%, BalancedAccuracy: 62.12%, GeometricMean: 57.23%, CohenKappa: 26.49%
[5,000] Accuracy: 72.85%, BalancedAccuracy: 63.23%, GeometricMean: 58.61%, CohenKappa: 28.94%
[6,000] Accuracy: 73.33%, BalancedAccuracy: 64.41%, GeometricMean: 60.49%, CohenKappa: 31.15%
[7,000] Accuracy: 73.91%, BalancedAccuracy: 65.10%, GeometricMean: 61.40%, CohenKappa: 32.57%
[8,000] Accuracy: 73.51%, BalancedAccuracy: 64.66%, GeometricMean: 60.77%, CohenKappa: 31.71%
[9,000] Accuracy: 73.81%, BalancedAccuracy: 64.68%, GeometricMean: 60.44%, CohenKappa: 32.08%
[10,000] Accuracy: 73.85%, BalancedAccuracy: 64.78%, GeometricMean: 60.66%, CohenKappa: 32.20%
[11,000] Accuracy: 74.03%, BalancedAccuracy: 64.61%, Geomet

Accuracy: 73.73%, BalancedAccuracy: 66.59%, GeometricMean: 63.77%, CohenKappa: 35.22%

## Concept Drift Impact

Concept drift can negatively impact learning methods if not properly handled. Multiple real-world applications suffer **model degradation** as the models can not adapt to changes in the data.

---
## `AGRAWAL` dataset

We will load the data from a csv file. The data was generated using the `AGRAWAL` data generator with 3 **gradual drifts** at the 5k, 10k, and 15k marks. It contains 9 features, 6 numeric and 3 categorical.

There are 10 functions for generating binary class labels from the features. These functions determine whether a **loan** should be approved.

| Feature    | Description            | Values                                                                |
|------------|------------------------|-----------------------------------------------------------------------|
| `salary`     | salary                 | uniformly distributed from 20k to 150k                                |
| `commission` | commission             | if (salary <   75k) then 0 else uniformly distributed from 10k to 75k |
| `age`        | age                    | uniformly distributed from 20 to 80                                   |
| `elevel`     | education level        | uniformly chosen from 0 to 4                                          |
| `car`        | car maker              | uniformly chosen from 1 to 20                                         |
| `zipcode`    | zip code of the town   | uniformly chosen from 0 to 8                                          |
| `hvalue`     | value of the house     | uniformly distributed from 50k x zipcode to 100k x zipcode            |
| `hyears`     | years house owned      | uniformly distributed from 1 to 30                                    |
| `loan`       | total loan amount      | uniformly distributed from 0 to 500k                                  |

**Class:** `y` | 0: no loan, 1: loan
 
**Samples:** 20,000

`elevel`, `car`, and `zipcode` are categorical features.

In [8]:
data = pd.read_csv("../datasets/agr_a_20k.csv")
features = data.columns[:-1]

## Naïve Bayes

In [9]:
from river.naive_bayes import GaussianNB

model = GaussianNB()
metrics = Rolling(metric=Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()]),window_size=500)
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=500)

[500] Accuracy: 81.76%, BalancedAccuracy: 74.69%, GeometricMean: 71.56%, CohenKappa: 54.66%	(rolling 500)
[1,000] Accuracy: 86.20%, BalancedAccuracy: 79.37%, GeometricMean: 76.72%, CohenKappa: 65.37%	(rolling 500)
[1,500] Accuracy: 87.80%, BalancedAccuracy: 82.87%, GeometricMean: 81.07%, CohenKappa: 71.19%	(rolling 500)
[2,000] Accuracy: 89.40%, BalancedAccuracy: 84.41%, GeometricMean: 82.96%, CohenKappa: 74.45%	(rolling 500)
[2,500] Accuracy: 89.80%, BalancedAccuracy: 84.49%, GeometricMean: 83.16%, CohenKappa: 74.70%	(rolling 500)
[3,000] Accuracy: 87.00%, BalancedAccuracy: 80.54%, GeometricMean: 78.15%, CohenKappa: 67.64%	(rolling 500)
[3,500] Accuracy: 87.60%, BalancedAccuracy: 79.87%, GeometricMean: 77.29%, CohenKappa: 67.25%	(rolling 500)
[4,000] Accuracy: 90.80%, BalancedAccuracy: 85.44%, GeometricMean: 84.19%, CohenKappa: 76.91%	(rolling 500)
[4,500] Accuracy: 87.00%, BalancedAccuracy: 80.91%, GeometricMean: 78.70%, CohenKappa: 68.03%	(rolling 500)
[5,000] Accuracy: 86.80%, Bala

Accuracy: 57.80%, BalancedAccuracy: 60.46%, GeometricMean: 59.40%, CohenKappa: 18.97%	(rolling 500)

## KNN with ADWIN
---

This classifier is an improvement from the regular kNN method, as it is resistant to concept drift. It uses the ADWIN change detector to decide which samples to keep and which ones to forget, and by doing so it regulates the sample window size.

In [11]:
from river.neighbors import KNNADWINClassifier
from river import compose

model = (
    compose.Discard('elevel', 'car', 'zipcode') |
    KNNADWINClassifier(n_neighbors=5, window_size=1000)
)
metrics = Rolling(metric=Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()]),window_size=500)
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=500)

[500] Accuracy: 58.92%, BalancedAccuracy: 50.82%, GeometricMean: 44.54%, CohenKappa: 1.76%	(rolling 500)
[1,000] Accuracy: 57.40%, BalancedAccuracy: 48.27%, GeometricMean: 39.88%, CohenKappa: -3.74%	(rolling 500)
[1,500] Accuracy: 58.00%, BalancedAccuracy: 50.18%, GeometricMean: 42.20%, CohenKappa: 0.39%	(rolling 500)
[2,000] Accuracy: 58.00%, BalancedAccuracy: 50.93%, GeometricMean: 45.88%, CohenKappa: 1.94%	(rolling 500)
[2,500] Accuracy: 61.00%, BalancedAccuracy: 51.18%, GeometricMean: 43.12%, CohenKappa: 2.58%	(rolling 500)
[3,000] Accuracy: 59.00%, BalancedAccuracy: 49.97%, GeometricMean: 41.91%, CohenKappa: -0.07%	(rolling 500)
[3,500] Accuracy: 63.00%, BalancedAccuracy: 52.91%, GeometricMean: 45.92%, CohenKappa: 6.28%	(rolling 500)
[4,000] Accuracy: 61.20%, BalancedAccuracy: 50.19%, GeometricMean: 40.28%, CohenKappa: 0.41%	(rolling 500)
[4,500] Accuracy: 61.00%, BalancedAccuracy: 51.86%, GeometricMean: 43.53%, CohenKappa: 4.10%	(rolling 500)
[5,000] Accuracy: 62.40%, BalancedAcc

Accuracy: 73.60%, BalancedAccuracy: 71.44%, GeometricMean: 70.86%, CohenKappa: 43.41%	(rolling 500)

## Hoeffding Tree

In [12]:
from river.tree import HoeffdingTreeClassifier

model = HoeffdingTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode'])
metrics = Rolling(metric=Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()]),window_size=500)
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream,
                      model=model,
                      metric=metrics,
                      print_every=500)

[500] Accuracy: 79.76%, BalancedAccuracy: 72.89%, GeometricMean: 69.86%, CohenKappa: 50.18%	(rolling 500)
[1,000] Accuracy: 83.60%, BalancedAccuracy: 75.91%, GeometricMean: 72.37%, CohenKappa: 58.36%	(rolling 500)
[1,500] Accuracy: 78.80%, BalancedAccuracy: 76.25%, GeometricMean: 75.74%, CohenKappa: 53.18%	(rolling 500)
[2,000] Accuracy: 87.00%, BalancedAccuracy: 90.15%, GeometricMean: 89.61%, CohenKappa: 73.49%	(rolling 500)
[2,500] Accuracy: 83.20%, BalancedAccuracy: 87.61%, GeometricMean: 86.73%, CohenKappa: 66.16%	(rolling 500)
[3,000] Accuracy: 88.60%, BalancedAccuracy: 91.44%, GeometricMean: 91.04%, CohenKappa: 76.38%	(rolling 500)
[3,500] Accuracy: 89.00%, BalancedAccuracy: 91.51%, GeometricMean: 91.28%, CohenKappa: 76.29%	(rolling 500)
[4,000] Accuracy: 91.40%, BalancedAccuracy: 92.69%, GeometricMean: 92.63%, CohenKappa: 81.10%	(rolling 500)
[4,500] Accuracy: 89.80%, BalancedAccuracy: 91.43%, GeometricMean: 91.29%, CohenKappa: 78.43%	(rolling 500)
[5,000] Accuracy: 90.80%, Bala

Accuracy: 67.20%, BalancedAccuracy: 66.57%, GeometricMean: 66.51%, CohenKappa: 32.29%	(rolling 500)

## Hoeffding Adaptive Tree

In [13]:
from river.tree import HoeffdingAdaptiveTreeClassifier

model = HoeffdingAdaptiveTreeClassifier(nominal_attributes=['elevel', 'car', 'zipcode'], seed=42)
metrics = Rolling(metric=Metrics(metrics=[Accuracy(),BalancedAccuracy(),GeometricMean(),CohenKappa()]),window_size=500)
stream = iter_pandas(X=data[features], y=data['class'])

progressive_val_score(dataset=stream, 
                      model=model, 
                      metric=metrics, 
                      print_every=500)

[500] Accuracy: 81.16%, BalancedAccuracy: 74.83%, GeometricMean: 72.34%, CohenKappa: 54.00%	(rolling 500)
[1,000] Accuracy: 85.40%, BalancedAccuracy: 80.74%, GeometricMean: 79.54%, CohenKappa: 65.13%	(rolling 500)
[1,500] Accuracy: 90.80%, BalancedAccuracy: 92.10%, GeometricMean: 91.99%, CohenKappa: 80.76%	(rolling 500)
[2,000] Accuracy: 91.80%, BalancedAccuracy: 92.79%, GeometricMean: 92.74%, CohenKappa: 82.41%	(rolling 500)
[2,500] Accuracy: 90.60%, BalancedAccuracy: 92.74%, GeometricMean: 92.55%, CohenKappa: 79.88%	(rolling 500)
[3,000] Accuracy: 92.20%, BalancedAccuracy: 93.55%, GeometricMean: 93.46%, CohenKappa: 83.24%	(rolling 500)
[3,500] Accuracy: 93.00%, BalancedAccuracy: 94.40%, GeometricMean: 94.33%, CohenKappa: 84.39%	(rolling 500)
[4,000] Accuracy: 95.20%, BalancedAccuracy: 96.49%, GeometricMean: 96.43%, CohenKappa: 89.33%	(rolling 500)
[4,500] Accuracy: 92.20%, BalancedAccuracy: 93.10%, GeometricMean: 93.05%, CohenKappa: 83.18%	(rolling 500)
[5,000] Accuracy: 92.80%, Bala

Accuracy: 77.60%, BalancedAccuracy: 79.18%, GeometricMean: 78.90%, CohenKappa: 55.16%	(rolling 500)

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) Quantia Consulting, srl. All rights reserved.